# Filter Data - WHERE

> 🔗 You can see more detailed info at SQLModel's [official docs](https://sqlmodel.tiangolo.com/tutorial/where/)

We will be using the same "Heroes" example from the previous notebooks. Make sure to delete the `database.db` file to start with a clean slate!

## Why filter data?

In the previous notebooks, were we querying tables and returning _ALL_ the data. With a big database, that could be a very expensive query (ie money and time), so it's more common to add filters to your queries to return only the rows of data you really need.

> Run the next cell to get our initial code defined and our database created with heroes

In [1]:
from typing import Optional

from sqlmodel import Field, Session, SQLModel, create_engine, select


class Hero(SQLModel, table=True):
    id: Optional[int] = Field(default=None, primary_key=True)
    name: str
    secret_name: str
    age: Optional[int] = None


sqlite_file_name = "database.db"
sqlite_url = f"sqlite:///{sqlite_file_name}"
engine = create_engine(sqlite_url, echo=True)


def create_db_and_tables():
    SQLModel.metadata.create_all(engine)


def create_heroes():
    hero_1 = Hero(name="Deadpond", secret_name="Dive Wilson")
    hero_2 = Hero(name="Spider-Boy", secret_name="Pedro Parqueador")
    hero_3 = Hero(name="Rusty-Man", secret_name="Tommy Sharp", age=48)

    with Session(engine) as session:
        session.add(hero_1)
        session.add(hero_2)
        session.add(hero_3)
        session.commit()


def main():
    create_db_and_tables()
    create_heroes()

main()

2023-12-14 17:28:42,449 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-14 17:28:42,449 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("hero")
2023-12-14 17:28:42,450 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-12-14 17:28:42,450 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("hero")
2023-12-14 17:28:42,451 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-12-14 17:28:42,451 INFO sqlalchemy.engine.Engine 
CREATE TABLE hero (
	id INTEGER NOT NULL, 
	name VARCHAR NOT NULL, 
	secret_name VARCHAR NOT NULL, 
	age INTEGER, 
	PRIMARY KEY (id)
)


2023-12-14 17:28:42,452 INFO sqlalchemy.engine.Engine [no key 0.00027s] ()
2023-12-14 17:28:42,453 INFO sqlalchemy.engine.Engine COMMIT
2023-12-14 17:28:42,454 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-14 17:28:42,456 INFO sqlalchemy.engine.Engine INSERT INTO hero (name, secret_name, age) VALUES (?, ?, ?) RETURNING id
2023-12-14 17:28:42,456 INFO sqlalchemy.engine.Engine [generated in 0.00009s (insertmanyvalues) 1/3 

## Filter Data with SQL

```sql
SELECT id, name, secret_name, age
FROM hero
WHERE name = "Deadpond"
```

The `WHERE` keyword (aka clause) adds a filter. The SQL query now reads a bit differently:

> Select all columns from the `hero` table and return only the rows where name equals `"Deadpond"`

Give this a try in DB Browser and see what you get!

### Challenge

Write a query that only returns the `name` and `age` columns from heroes that are older than 20.

### Good to know

* `SELECT` tells the databse which **columns** to return
* `WHERE` tells the database which **rows** to return

The size of the table in the two dimensions depend mostly on those two keywords.


## Filter Data with SQLModel

Now, the same way that we add `WHERE` to a SQL statement to filter rows, we can add a `.where()` to a SQLModel `select()`` statement to filter rows, which will filter the objects returned:

In [2]:
def select_heroes():
    with Session(engine) as session:
        statement = select(Hero).where(Hero.name == "Deadpond")
        heroes = session.exec(statement).all()
        return heroes

select_heroes()

2023-12-14 17:28:50,259 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-14 17:28:50,264 INFO sqlalchemy.engine.Engine SELECT hero.id, hero.name, hero.secret_name, hero.age 
FROM hero 
WHERE hero.name = ?
2023-12-14 17:28:50,265 INFO sqlalchemy.engine.Engine [generated in 0.00137s] ('Deadpond',)
2023-12-14 17:28:50,267 INFO sqlalchemy.engine.Engine ROLLBACK


[Hero(secret_name='Dive Wilson', age=None, id=1, name='Deadpond')]

### Chain `select()` objects

To add multiple `where()` clauses, you can simply chain them together. For example, what do you think this function would return:

In [3]:
def select_heroes():
    with Session(engine) as session:
        # Notice how we can use regular Python operators in the where clauses
        statement = select(Hero).where(Hero.name != "Deadpond").where(Hero.age > 20)
        heroes = session.exec(statement).all()
        return heroes

select_heroes()

2023-12-14 17:28:53,022 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-14 17:28:53,025 INFO sqlalchemy.engine.Engine SELECT hero.id, hero.name, hero.secret_name, hero.age 
FROM hero 
WHERE hero.name != ? AND hero.age > ?
2023-12-14 17:28:53,026 INFO sqlalchemy.engine.Engine [generated in 0.00105s] ('Deadpond', 20)
2023-12-14 17:28:53,028 INFO sqlalchemy.engine.Engine ROLLBACK


[Hero(secret_name='Tommy Sharp', age=48, id=3, name='Rusty-Man')]

### Refactor our query function

Our `select_heroes` function has `"Deadpond"` hard-coded, but what if we wanted to reuse the same function to get different heroes?

Instead of writing a new function for each name, we could use parameters:

In [4]:
# (optional, but recommended) I've added type hints to the function signature
def select_heroes_by_name(name: str) -> list[Hero]:
    with Session(engine) as session:
        # Notice how we are using the parameter in the where clause
        statement = select(Hero).where(Hero.name == name)
        heroes = session.exec(statement).all()
        return heroes

select_heroes_by_name(name="Spider-Boy")

2023-12-14 17:28:59,119 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-14 17:28:59,121 INFO sqlalchemy.engine.Engine SELECT hero.id, hero.name, hero.secret_name, hero.age 
FROM hero 
WHERE hero.name = ?
2023-12-14 17:28:59,122 INFO sqlalchemy.engine.Engine [cached since 8.858s ago] ('Spider-Boy',)
2023-12-14 17:28:59,123 INFO sqlalchemy.engine.Engine ROLLBACK


[Hero(secret_name='Pedro Parqueador', age=None, id=2, name='Spider-Boy')]

## Add more data

To continue exploring and experimenting with `SELECT` and `WHERE`, let's add some more data.

In [5]:
def create_heroes():
    hero_1 = Hero(name="Deadpond", secret_name="Dive Wilson")
    hero_2 = Hero(name="Spider-Boy", secret_name="Pedro Parqueador")
    hero_3 = Hero(name="Rusty-Man", secret_name="Tommy Sharp", age=48)
    hero_4 = Hero(name="Tarantula", secret_name="Natalia Roman-on", age=32)
    hero_5 = Hero(name="Black Lion", secret_name="Trevor Challa", age=35)
    hero_6 = Hero(name="Dr. Weird", secret_name="Steve Weird", age=36)
    hero_7 = Hero(name="Captain North America", secret_name="Esteban Rogelios", age=93)

    with Session(engine) as session:
        session.add(hero_1)
        session.add(hero_2)
        session.add(hero_3)
        session.add(hero_4)
        session.add(hero_5)
        session.add(hero_6)
        session.add(hero_7)

        session.commit()

create_heroes()

2023-12-14 17:29:01,693 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-14 17:29:01,695 INFO sqlalchemy.engine.Engine INSERT INTO hero (name, secret_name, age) VALUES (?, ?, ?) RETURNING id
2023-12-14 17:29:01,696 INFO sqlalchemy.engine.Engine [cached since 19.24s ago (insertmanyvalues) 1/7 (ordered; batch not supported)] ('Deadpond', 'Dive Wilson', None)
2023-12-14 17:29:01,698 INFO sqlalchemy.engine.Engine INSERT INTO hero (name, secret_name, age) VALUES (?, ?, ?) RETURNING id
2023-12-14 17:29:01,704 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/7 (ordered; batch not supported)] ('Spider-Boy', 'Pedro Parqueador', None)
2023-12-14 17:29:01,708 INFO sqlalchemy.engine.Engine INSERT INTO hero (name, secret_name, age) VALUES (?, ?, ?) RETURNING id
2023-12-14 17:29:01,712 INFO sqlalchemy.engine.Engine [insertmanyvalues 3/7 (ordered; batch not supported)] ('Rusty-Man', 'Tommy Sharp', 48)
2023-12-14 17:29:01,717 INFO sqlalchemy.engine.Engine INSERT INTO hero (name, secret_name, ag

In [6]:
# Let's see all the heroes in the DB
def select_all_heroes():
    with Session(engine) as session:
        statement = select(Hero)
        heroes = session.exec(statement).all()
        return heroes

select_all_heroes()

2023-12-14 17:29:07,441 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-14 17:29:07,443 INFO sqlalchemy.engine.Engine SELECT hero.id, hero.name, hero.secret_name, hero.age 
FROM hero
2023-12-14 17:29:07,444 INFO sqlalchemy.engine.Engine [generated in 0.00113s] ()
2023-12-14 17:29:07,447 INFO sqlalchemy.engine.Engine ROLLBACK


[Hero(secret_name='Dive Wilson', age=None, id=1, name='Deadpond'),
 Hero(secret_name='Pedro Parqueador', age=None, id=2, name='Spider-Boy'),
 Hero(secret_name='Tommy Sharp', age=48, id=3, name='Rusty-Man'),
 Hero(secret_name='Dive Wilson', age=None, id=4, name='Deadpond'),
 Hero(secret_name='Pedro Parqueador', age=None, id=5, name='Spider-Boy'),
 Hero(secret_name='Tommy Sharp', age=48, id=6, name='Rusty-Man'),
 Hero(secret_name='Natalia Roman-on', age=32, id=7, name='Tarantula'),
 Hero(secret_name='Trevor Challa', age=35, id=8, name='Black Lion'),
 Hero(secret_name='Steve Weird', age=36, id=9, name='Dr. Weird'),
 Hero(secret_name='Esteban Rogelios', age=93, id=10, name='Captain North America')]

### How are there multiple Deadponds now??

There are some duplicates, but this is expected because it's the `Primary Key` that serves as the unique identifier, not the hero's name.

When we defined the `Hero` class, we marked the `id` as the Primary Key.

In [7]:
select_heroes_by_name("Deadpond")

2023-12-14 17:29:11,305 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-14 17:29:11,307 INFO sqlalchemy.engine.Engine SELECT hero.id, hero.name, hero.secret_name, hero.age 
FROM hero 
WHERE hero.name = ?
2023-12-14 17:29:11,309 INFO sqlalchemy.engine.Engine [cached since 21.05s ago] ('Deadpond',)
2023-12-14 17:29:11,310 INFO sqlalchemy.engine.Engine ROLLBACK


[Hero(secret_name='Dive Wilson', age=None, id=1, name='Deadpond'),
 Hero(secret_name='Dive Wilson', age=None, id=4, name='Deadpond')]

## Comparisons

* `==` is equal to
* `!=` is not equal to
* `>` is greater than
* `>=` is greater than or equal to
* `<` is less than
* `<=` is less than or equal to

## Multiple WHERE clauses - AND

You can chain `WHERE` clauses, but sometimes your filter requires that that you _compound_ the clauses. For example, look at this SQL query:

```sql
SELECT id, name, secret_name, age
FROM hero
WHERE age >= 35 AND age < 40
```

The `AND` keyword adds more constraints to this filter. With this, the database will return heroes that are 35-39 years old!

With **SQLModel**, you can do this a few ways:

In [ ]:
def where_with_chains():
    with Session(engine) as session:
        statement = select(Hero).where(Hero.age >= 35).where(Hero.age < 40)
        results = session.exec(statement)
        for hero in results:
            print(hero)

def where_with_expressions():
    with Session(engine) as session:
        statement = select(Hero).where(Hero.age >= 35, Hero.age < 40)
        results = session.exec(statement)
        for hero in results:
            print(hero)

## Multiple WHERE clauses - OR

Similar to `AND`, your filters may require the `OR` keyword:

```sql
SELECT id, name, secret_name, age
FROM hero
WHERE age <= 35 OR age > 90
```

With **SQLModel**, you can import `or_`:

In [ ]:
from sqlmodel import or_

def select_heroes():
    with Session(engine) as session:
        statement = select(Hero).where(or_(Hero.age <= 35, Hero.age > 90))
        results = session.exec(statement)
        for hero in results:
            print(hero)